##  Treoubleshooting workflows

This is similar to the advanced notebook on using workflows but shows how to access workflow outputs in napari-lattice

In [1]:
import os
import pyclesperanto_prototype as cle
from napari_workflows import Workflow
from napari_workflows import _io_yaml_v1 as io_yaml
from lls_core import LatticeData
import numpy as np
import pandas as pd 

cle.get_device()


c:\Users\Pradeep\.conda\envs\napari-lattice\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


<NVIDIA GeForce RTX 3080 on Platform: NVIDIA CUDA (1 refs)>

In [2]:
#download sample data from https://zenodo.org/records/14903188
img_path = "./sample_data/RBC_tiny.czi"

#make dir if not exists
if not os.path.exists("./sample_data"):
    os.makedirs("./sample_data")

import urllib.request

url = "https://zenodo.org/records/14903188/files/RBC_tiny.czi?download=1"
if not os.path.exists(img_path):
    print("Downloading file")
    urllib.request.urlretrieve(url, img_path)
    print(f"Downloaded at : {img_path}")
else:
    print(f"File already exists at : {img_path}")
    
#absolute path for saving
#get current path
cur_dir = os.getcwd()
save_path = os.path.join(cur_dir, "output")

#make dir if not exists
if not os.path.exists(save_path):
    os.makedirs(save_path)

File already exists at : ./sample_data/RBC_tiny.czi


We have a test workflow, which applies: 
- gaussian
- binarisation
- labeling


We would like to save the deskewed image, segmented image and the measurements as a csv file, so we return all 3.

In [3]:
def image_workflow() -> Workflow:
    # Simple segmentation workflow that returns an image
    image_seg_workflow = Workflow()
    image_seg_workflow.set("gaussian", cle.gaussian_blur, "deskewed_image", sigma_x=1, sigma_y=1, sigma_z=1)
    image_seg_workflow.set("binarisation", cle.threshold, "gaussian", constant=0.5)
    image_seg_workflow.set("labeling", cle.connected_components_labeling_box, "binarisation")
    image_seg_workflow.set("result",measure_region,"deskewed_image","labeling")
    return image_seg_workflow

def measure_region(image,label) -> dict:
    # Measure properties of the segmented regions
    from skimage.measure import regionprops_table
    #convert to numpy
    image = np.asarray(image)
    label = np.asarray(label)
    props = regionprops_table(label, image,
                              properties=['label', 'bbox','area', 
                                          'intensity_mean'])
    return props

image_analysis_workflow = image_workflow()

print(image_analysis_workflow)

Workflow:
gaussian <- (<function gaussian_blur at 0x000002D8D2D1CA60>, 'deskewed_image', None, 1, 1, 1)
binarisation <- (<function greater_constant at 0x000002D8D2D1D360>, 'gaussian', None, 0.5)
labeling <- (<function connected_components_labeling_box at 0x000002D8D2E05CF0>, 'binarisation')
result <- (<function measure_region at 0x000002D8A449EDD0>, 'deskewed_image', 'labeling')



In [4]:
params_workflow = LatticeData(
  input_image=img_path,
  save_dir=save_path,
  workflow=image_analysis_workflow,
)
params_workflow

[INFO:2025-09-02 14:51:29,318] Processing File ./sample_data/RBC_tiny.czi


LatticeData(input_image=<xarray.DataArray 'transpose-aa2bd2dc5da500b61f8f6aa8fdb114ba' (T: 1, C: 1,
                                                                Z: 834, Y: 118,
                                                                X: 209)>
dask.array<transpose, shape=(1, 1, 834, 118, 209), dtype=uint16, chunksize=(1, 1, 834, 118, 209), chunktype=numpy.ndarray>
Coordinates:
  * C        (C) <U17 'LatticeLightsheet'
  * Z        (Z) float64 0.0 0.3 0.6 0.9 1.2 ... 248.7 249.0 249.3 249.6 249.9
  * Y        (Y) float64 0.0 0.145 0.29 0.435 0.58 ... 16.53 16.67 16.82 16.96
  * X        (X) float64 0.0 0.145 0.29 0.435 0.58 ... 29.72 29.87 30.01 30.16
Dimensions without coordinates: T
Attributes:
    unprocessed:  <Element 'ImageDocument' at 0x000002D8892E6B10>, skew=<DeskewDirection.Y: 2>, angle=30.0, physical_pixel_sizes=DefinedPixelSizes(X=0.14499219272808386, Y=0.14499219272808386, Z=0.3), derived=DerivedDeskewFields(deskew_vol_shape=(59, 1828, 209), deskew_affine_transform

Troubleshooting

To inspect the components of workflows, you can use 2 approaches:
- `process_workflow` to access the output from the workflow
- `process_workflow().process()` to access the workflow output after metadata, such as time and channel has been added to it. 

`process_workflow`: It returns a generator object, which can be accessed using the `next` function



In [5]:
for slice in params_workflow.process_workflow():
    first_slice = slice
    break
first_slice

('slices',
 <generator object LatticeData.process_workflow.<locals>._generator at 0x000002D88929D540>)

In [6]:
data_set = next(first_slice[1])
print(len(data_set.data)) #1

Timepoints:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO:2025-09-02 14:51:29,827] Processing File <xarray.DataArray 'transpose-aa2bd2dc5da500b61f8f6aa8fdb114ba' (Z: 834, Y: 118,
                                                                X: 209)>
dask.array<getitem, shape=(834, 118, 209), dtype=uint16, chunksize=(834, 118, 209), chunktype=numpy.ndarray>
Coordinates:
    C        <U17 'LatticeLightsheet'
  * Z        (Z) float64 0.0 0.3 0.6 0.9 1.2 ... 248.7 249.0 249.3 249.6 249.9
  * Y        (Y) float64 0.0 0.145 0.29 0.435 0.58 ... 16.53 16.67 16.82 16.96
  * X        (X) float64 0.0 0.145 0.29 0.435 0.58 ... 29.72 29.87 30.01 30.16
Attributes:
    unprocessed:  <Element 'ImageDocument' at 0x000002D8892E6B10>


1


In [7]:
data_set.data

({'label': array([1]),
  'bbox-0': array([0]),
  'bbox-1': array([0]),
  'bbox-2': array([0]),
  'bbox-3': array([59]),
  'bbox-4': array([1827]),
  'bbox-5': array([209]),
  'area': array([21393187.]),
  'intensity_mean': array([321.0536499])},)

In [8]:
pd.DataFrame(data_set.data)

,label,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,area,intensity_mean
0,[1],[0],[0],[0],[59],[1827],[209],[21393187.0],[321.05364990234375]


In [9]:
outputs = []
for output in params_workflow.process_workflow().process():
    data = output.data
    outputs.append(data)
outputs

Timepoints:   0%|          | 0/1 [00:00<?, ?it/s][INFO:2025-09-02 14:51:40,603] Processing File <xarray.DataArray 'transpose-aa2bd2dc5da500b61f8f6aa8fdb114ba' (Z: 834, Y: 118,
                                                                X: 209)>
dask.array<getitem, shape=(834, 118, 209), dtype=uint16, chunksize=(834, 118, 209), chunktype=numpy.ndarray>
Coordinates:
    C        <U17 'LatticeLightsheet'
  * Z        (Z) float64 0.0 0.3 0.6 0.9 1.2 ... 248.7 249.0 249.3 249.6 249.9
  * Y        (Y) float64 0.0 0.145 0.29 0.435 0.58 ... 16.53 16.67 16.82 16.96
  * X        (X) float64 0.0 0.145 0.29 0.435 0.58 ... 29.72 29.87 30.01 30.16
Attributes:
    unprocessed:  <Element 'ImageDocument' at 0x000002D8892E6B10>
[INFO:2025-09-02 14:51:40,603] Processing File <xarray.DataArray 'transpose-aa2bd2dc5da500b61f8f6aa8fdb114ba' (Z: 834, Y: 118,
                                                                X: 209)>
dask.array<getitem, shape=(834, 118, 209), dtype=uint16, chunksize=(834, 118

[   time channel label bbox-0 bbox-1 bbox-2 bbox-3  bbox-4 bbox-5  \
 0  [T0]    [C0]   [1]    [0]    [0]    [0]   [59]  [1827]  [209]   
 
            area        intensity_mean  
 0  [21393187.0]  [321.05364990234375]  ]

In [11]:
outputs[0].apply(pd.Series.explode)

,time,channel,label,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,area,intensity_mean
0,T0,C0,1,0,0,0,59,1827,209,21393187.0,321.05365
